In [27]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

In [28]:
# File to Load 
school_data_to_load = Path("../PyCitySchools/Resources/schools_complete.csv")
student_data_to_load = Path("../PyCitySchools/Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [29]:
# Calculate the total number of unique schools
total_schools_unique = school_data_complete['school_name'].nunique()
print("Total number schools in the district")
total_schools_unique

Total number schools in the district


15

In [33]:
# Calculate the total number of students
Total_student_count = school_data_complete['student_name'].count()
print("Total students in the district")
Total_student_count

Total students in the district


39170

In [37]:
# Calculate the total budget
total_budget = school_data['budget'].sum()
print("Total budget in the district")
total_budget

Total budget in the district


24649428

In [38]:
# Calculate the average (mean) math score
average_math_score = school_data_complete['math_score'].mean()
print("Average Math score of the school district")
average_math_score

Average Math score of the school district


78.98537145774827

In [43]:
# Calculate the average (mean) reading score
average_reading_score =  school_data_complete['reading_score'].mean()
print("Average Math score of the school district")
average_reading_score

Average Math score of the school district


81.87784018381414

In [46]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(Total_student_count) * 100
passing_math_percentage

74.9808526933878

In [48]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(Total_student_count) * 100
passing_reading_percentage

85.80546336482001

In [56]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(Total_student_count) * 100
overall_passing_rate

65.17232575950983

In [136]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary=pd.DataFrame(school_data_complete)
district_summary=school_data_complete.drop(['Student ID', 'student_name', 'gender', 'School Name', 'reading_score', 'math_score', 
'School ID', 'Total School Budget', 'grade', 'School Type', 'Total Students'], axis=1)
# Formatting
district_summary["Total Student"] = Total_student_count
district_summary["Total Budget"] = total_budget
district_summary["Avarage Math Score"] = average_math_score
district_summary["%  Passing Math "] = passing_math_percentage
district_summary["%  Passing Reading"] = passing_reading_percentage
district_summary["% Overall Passing"] = overall_passing_rate
# Display the DataFrame
district_summary.head(1)


,Total Student,Total Budget,Avarage Math Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,24649428,78.985371,74.980853,85.805463,65.172326


## School Summary

Perform the necessary calculations and then create a DataFrame that summarizes key metrics about each school.
Including:

In [137]:
#School name

group_df = school_data_complete
school_data_complete=school_data_complete.rename(columns={"school_name":"School Name","type":"School Type","size":
"Total Students","budget":"Total School Budget"})


group_df=group_df.rename(columns={"school_name":"School Name","type":"School Type","size":"Total Students","budget":
"Total School Budget","reading_score":"Average Reading Score","math_score":"Average Math Score"})
group_df_final=group_df.drop(['School ID','Student ID'], axis=1)


group_df=group_df.reindex(columns=["School Name","School Type","Total Students","Total School Budget","Per Student Budget",
"Average Math Score","Average Reading Score"])


#del group_df['School ID','Student ID']
group_df = group_df.groupby(["School Name"])
group_average=group_df.mean(numeric_only= True)

per_school_summary=group_average

per_school_summary["Per Student Budget"] = group_average['Total School Budget'] / group_average['Total Students']
float_per_school_summary=per_school_summary["Per Student Budget"]
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].astype(float).map("${:,.2f}".format)
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].astype(float).map("${:,.2f}".format)


passing_math=school_data_complete.loc[school_data_complete['math_score']>=70, : ]
passing_reading=school_data_complete.loc[school_data_complete['reading_score']>=70, : ]
passing_math_count=pasing_math["School Name"].value_counts()
passing_reading_count=pasing_reading["School Name"].value_counts()


passing_math_count=pd.DataFrame(passing_math_count)
per_school_summary["passing math"]=passing_math_count

passing_reading_count=pd.DataFrame(passing_reading_count)
per_school_summary["passing reading"]=passing_reading_count

pass_all=school_data_complete.loc[(school_data_complete['math_score']>=70) & (school_data_complete['reading_score']>=70),:]
pass_all=pass_all["School Name"].value_counts()
pass_all=pd.DataFrame(pass_all)
per_school_summary["Passing Overall"]=pass_all

#school_data_complete = pd.concat([school_data_complete, pasing_math_count], ignore_index=True)
#school_data_complete["School Name"].value_counts()
per_school_summary["Passing Rate Math"]=(per_school_summary["passing math"]/per_school_summary["Total Students"])*100
per_school_summary["Passing Rate Reading"]=(per_school_summary["passing reading"]/per_school_summary["Total Students"])*100
per_school_summary["Passing Rate Overall"]=(per_school_summary["Passing Overall"]/per_school_summary["Total Students"])*100


per_school_summary["Passing Rate Reading"] = per_school_summary["Passing Rate Reading"].astype(float).map("{:,.2f}%".format)
per_school_summary["Passing Rate Math"] = per_school_summary["Passing Rate Math"].astype(float).map("{:,.2f}%".format)
per_school_summary["Passing Rate Overall"] = per_school_summary["Passing Rate Overall"].astype(float).map("{:,.2f}%".format)


#per_school_summary=pasing_math_count

per_school_summary=per_school_summary.drop(['passing math','passing reading','Passing Overall'], axis=1)
per_school_summary

group_df2=school_data_complete.reindex(columns=["School Name","School Type"])
#group_df2=group_df2["School Type"].value_counts()
group_df2=group_df2.drop_duplicates()

per_school_summary=pd.merge(per_school_summary, group_df2, how="left", on=["School Name", "School Name"])

first_column = per_school_summary.pop("School Type")
  
# insert column using insert(position,column_name,
# first_column) function
per_school_summary.insert(1, 'School Type', first_column)
#per_school_summary.insert(1, "School Type", group_df2["School Type"])
per_school_summary



,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Passing Rate Math,Passing Rate Reading,Passing Rate Overall
0,Bailey High School,District,4976.0,"$3,124,928.00",$628.00,77.048432,81.033963,66.68%,81.93%,54.64%
1,Cabrera High School,Charter,1858.0,"$1,081,356.00",$582.00,83.061895,83.975780,94.13%,97.04%,91.33%
2,Figueroa High School,District,2949.0,"$1,884,411.00",$639.00,76.711767,81.158020,65.99%,80.74%,53.20%
3,Ford High School,District,2739.0,"$1,763,916.00",$644.00,77.102592,80.746258,68.31%,79.30%,54.29%
4,Griffin High School,Charter,1468.0,"$917,500.00",$625.00,83.351499,83.816757,93.39%,97.14%,90.60%
5,Hernandez High School,District,4635.0,"$3,022,020.00",$652.00,77.289752,80.934412,66.75%,80.86%,53.53%
6,Holden High School,Charter,427.0,"$248,087.00",$581.00,83.803279,83.814988,92.51%,96.25%,89.23%
7,Huang High School,District,2917.0,"$1,910,635.00",$655.00,76.629414,81.182722,65.68%,81.32%,53.51%
8,Johnson High School,District,4761.0,"$3,094,650.00",$650.00,77.072464,80.966394,66.06%,81.22%,53.54%
9,Pena High School,Charter,962.0,"$585,858.00",$609.00,83.839917,84.044699,94.59%,95.95%,90.54%


## Highest-Performing Schools (by % Overall Passing)

In [84]:
#Sort the schools by % Overall Passing in descending order and display the top 5 rows.
#Save the results in a DataFrame called "top_schools".
top_schools = per_school_summary.sort_values('Passing Rate Overall', ascending=False)
top_schools.head(5)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Passing Rate Math,Passing Rate Reading,Passing Rate Overall
1,Cabrera High School,Charter,1858.0,"$1,081,356.00",$582.00,83.061895,83.975780,94.13%,97.04%,91.33%
12,Thomas High School,Charter,1635.0,"$1,043,130.00",$638.00,83.418349,83.848930,93.27%,97.31%,90.95%
4,Griffin High School,Charter,1468.0,"$917,500.00",$625.00,83.351499,83.816757,93.39%,97.14%,90.60%
13,Wilson High School,Charter,2283.0,"$1,319,574.00",$578.00,83.274201,83.989488,93.87%,96.54%,90.58%
9,Pena High School,Charter,962.0,"$585,858.00",$609.00,83.839917,84.044699,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

In [86]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values('Passing Rate Overall')
bottom_schools.head(5)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Passing Rate Math,Passing Rate Reading,Passing Rate Overall
10,Rodriguez High School,District,3999.0,"$2,547,363.00",$637.00,76.842711,80.744686,66.37%,80.22%,52.99%
2,Figueroa High School,District,2949.0,"$1,884,411.00",$639.00,76.711767,81.158020,65.99%,80.74%,53.20%
7,Huang High School,District,2917.0,"$1,910,635.00",$655.00,76.629414,81.182722,65.68%,81.32%,53.51%
5,Hernandez High School,District,4635.0,"$3,022,020.00",$652.00,77.289752,80.934412,66.75%,80.86%,53.53%
8,Johnson High School,District,4761.0,"$3,094,650.00",$650.00,77.072464,80.966394,66.06%,81.22%,53.54%


## Math Scores by Grade

In [87]:
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

ninth_graders = ninth_graders.groupby(["School Name"]).mean(numeric_only=True)
tenth_graders = tenth_graders.groupby(["School Name"]).mean(numeric_only=True)
eleventh_graders = eleventh_graders.groupby(["School Name"]).mean(numeric_only=True)
twelfth_graders = twelfth_graders.groupby(["School Name"]).mean(numeric_only=True)
grades_per_grade_math=pd.DataFrame()

grades_per_grade_math["9th"]=ninth_graders["math_score"]
grades_per_grade_math["10th"]=tenth_graders["math_score"]
grades_per_grade_math["11th"]=eleventh_graders["math_score"]
grades_per_grade_math["12th"]=twelfth_graders["math_score"]
grades_per_grade_math

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

In [88]:
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

ninth_graders = ninth_graders.groupby(["School Name"]).mean(numeric_only=True)
tenth_graders = tenth_graders.groupby(["School Name"]).mean(numeric_only=True)
eleventh_graders = eleventh_graders.groupby(["School Name"]).mean(numeric_only=True)
twelfth_graders = twelfth_graders.groupby(["School Name"]).mean(numeric_only=True)
grades_per_grade_reading=pd.DataFrame()

grades_per_grade_reading["9th"]=ninth_graders["reading_score"]
grades_per_grade_reading["10th"]=tenth_graders["reading_score"]
grades_per_grade_reading["11th"]=eleventh_graders["reading_score"]
grades_per_grade_reading["12th"]=twelfth_graders["reading_score"]
grades_per_grade_reading

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

In [90]:
spending_bins = [0, 585, 630, 645, 690]
labelz = ['$<585', "$585-630", "$630-645", "$645-680"]
school_spending_df = per_school_summary.copy()
school_spending_df['Per Student Budget'] = school_spending_df['Per Student Budget'].replace('[\$,]', '', regex=True).astype(float)
school_spending_df['Passing Rate Math'] = school_spending_df['Passing Rate Math'].replace('[\%,]', '', regex=True).astype(float)
school_spending_df['Passing Rate Reading'] = school_spending_df['Passing Rate Reading'].replace('[\%,]', '', regex=True).astype(float)
school_spending_df['Passing Rate Overall'] = school_spending_df['Passing Rate Overall'].replace('[\%,]', '', regex=True).astype(float)



school_spending_df['Spending Ranges (Per Student)']=pd.cut(school_spending_df['Per Student Budget'], bins=spending_bins, labels=labelz)
school_spending_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Passing Rate Math,Passing Rate Reading,Passing Rate Overall,Spending Ranges (Per Student)
0,Bailey High School,District,4976.0,"$3,124,928.00",628.0,77.048432,81.033963,66.68,81.93,54.64,$585-630
1,Cabrera High School,Charter,1858.0,"$1,081,356.00",582.0,83.061895,83.975780,94.13,97.04,91.33,$<585
2,Figueroa High School,District,2949.0,"$1,884,411.00",639.0,76.711767,81.158020,65.99,80.74,53.20,$630-645
3,Ford High School,District,2739.0,"$1,763,916.00",644.0,77.102592,80.746258,68.31,79.30,54.29,$630-645
4,Griffin High School,Charter,1468.0,"$917,500.00",625.0,83.351499,83.816757,93.39,97.14,90.60,$585-630
5,Hernandez High School,District,4635.0,"$3,022,020.00",652.0,77.289752,80.934412,66.75,80.86,53.53,$645-680
6,Holden High School,Charter,427.0,"$248,087.00",581.0,83.803279,83.814988,92.51,96.25,89.23,$<585
7,Huang High School,District,2917.0,"$1,910,635.00",655.0,76.629414,81.182722,65.68,81.32,53.51,$645-680
8,Johnson High School,District,4761.0,"$3,094,650.00",650.0,77.072464,80.966394,66.06,81.22,53.54,$645-680
9,Pena High School,Charter,962.0,"$585,858.00",609.0,83.839917,84.044699,94.59,95.95,90.54,$585-630


In [91]:
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_math_scores=pd.DataFrame(spending_math_scores)

spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_reading_scores=pd.DataFrame(spending_reading_scores)

spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Passing Rate Math"].mean()
spending_passing_math=pd.DataFrame(spending_passing_math)

spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Passing Rate Reading"].mean()
spending_passing_reading=pd.DataFrame(spending_passing_reading)

overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Passing Rate Overall"].mean()
overall_passing_spending=pd.DataFrame(overall_passing_spending)

spending_summary = pd.merge(spending_math_scores, spending_reading_scores, how="left", on=["Spending Ranges (Per Student)"])
spending_summary=pd.merge(spending_summary, spending_passing_math, how="left", on=["Spending Ranges (Per Student)"])

spending_summary=pd.merge(spending_summary, spending_passing_reading, how="left", on=["Spending Ranges (Per Student)"])
spending_summary=pd.merge(spending_summary, overall_passing_spending, how="left", on=["Spending Ranges (Per Student)"])
spending_summary

,Average Math Score,Average Reading Score,Passing Rate Math,Passing Rate Reading,Passing Rate Overall
Spending Ranges (Per Student),,,,,
$<585,83.455399,83.933814,93.460000,96.610000,90.367500
$585-630,81.899826,83.155286,87.132500,92.717500,81.417500
$630-645,78.518855,81.624473,73.485000,84.392500,62.857500
$645-680,76.997210,81.027843,66.163333,81.133333,53.526667


In [92]:
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

per_school_summary['School Size']=pd.cut(per_school_summary['Total Students'], bins=size_bins, labels=labels)

per_school_summary

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Passing Rate Math,Passing Rate Reading,Passing Rate Overall,School Size
0,Bailey High School,District,4976.0,"$3,124,928.00",$628.00,77.048432,81.033963,66.68%,81.93%,54.64%,Large (2000-5000)
1,Cabrera High School,Charter,1858.0,"$1,081,356.00",$582.00,83.061895,83.975780,94.13%,97.04%,91.33%,Medium (1000-2000)
2,Figueroa High School,District,2949.0,"$1,884,411.00",$639.00,76.711767,81.158020,65.99%,80.74%,53.20%,Large (2000-5000)
3,Ford High School,District,2739.0,"$1,763,916.00",$644.00,77.102592,80.746258,68.31%,79.30%,54.29%,Large (2000-5000)
4,Griffin High School,Charter,1468.0,"$917,500.00",$625.00,83.351499,83.816757,93.39%,97.14%,90.60%,Medium (1000-2000)
5,Hernandez High School,District,4635.0,"$3,022,020.00",$652.00,77.289752,80.934412,66.75%,80.86%,53.53%,Large (2000-5000)
6,Holden High School,Charter,427.0,"$248,087.00",$581.00,83.803279,83.814988,92.51%,96.25%,89.23%,Small (<1000)
7,Huang High School,District,2917.0,"$1,910,635.00",$655.00,76.629414,81.182722,65.68%,81.32%,53.51%,Large (2000-5000)
8,Johnson High School,District,4761.0,"$3,094,650.00",$650.00,77.072464,80.966394,66.06%,81.22%,53.54%,Large (2000-5000)
9,Pena High School,Charter,962.0,"$585,858.00",$609.00,83.839917,84.044699,94.59%,95.95%,90.54%,Small (<1000)


In [186]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["Average Math Score"]).mean(numeric_only=True)
size_math_scores= pd.DataFrame(size_math_scores)

size_reading_scores = per_school_summary.groupby(["Average Reading Score"]).mean(numeric_only=True)
size_reading_scores= pd.DataFrame(size_reading_scores)

size_passing_math = per_school_summary.groupby(["Passing Rate Math"]).mean(numeric_only=True)
size_passing_math= pd.DataFrame(size_passing_math)

size_passing_reading = per_school_summary.groupby(["Passing Rate Reading"]).mean(numeric_only=True)
size_passing_reading= pd.DataFrame(size_passing_reading)

size_overall_passing = per_school_summary.groupby(["Passing Rate Overall"]).mean(numeric_only=True)
size_overall_passing= pd.DataFrame(size_overall_passing)


size_summary = pd.merge(size_summary,size_math_scores, how="left", on=["School Size"])

size_summary 

KeyError: 'School Size'

In [182]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame(per_school_summary.groupby["School Size"])


# Display results
size_summary

TypeError: 'method' object is not subscriptable